In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from keras.utils import to_categorical
from keras.optimizers import SGD
from sklearn.utils import shuffle

Using TensorFlow backend.


In [10]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '17uc0OhmfLfI5P94a8ytlQ42C6PUcBS1t'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('A_Z_HandwrittenData.csv')

print('Done...')

Done...


In [4]:
X = pd.read_csv('A_Z_HandwrittenData.csv', header=0)
X = shuffle(X)
X.columns = ['label'] + ['Pixel{}'.format(i) for i in range(784)]

X.head()

,label,Pixel0,Pixel1,Pixel2,Pixel3,Pixel4,Pixel5,Pixel6,Pixel7,Pixel8,...,Pixel774,Pixel775,Pixel776,Pixel777,Pixel778,Pixel779,Pixel780,Pixel781,Pixel782,Pixel783
345128,22,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102201,11,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
165056,14,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
307640,19,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9043,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
Y = to_categorical(X.label.values, num_classes=26)
X = X[['Pixel{}'.format(i) for i in range(784)]].values
X = X.reshape(-1, 28, 28, 1)

X_train = X[:300000]
X_test = X[300000:]
del X

Y_train = Y[:300000]
Y_test = Y[300000:]
del Y

print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(300000, 28, 28, 1) (300000, 26) (72037, 28, 28, 1) (72037, 26)


In [0]:
from scipy.misc import imsave

X = X_train.reshape(-1, 28, 28)
for i in range(100):
    imsave('img/images_{}.jpg'.format(i), X[i, :, :])

del X

In [12]:
#The Model
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='SAME', activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='SAME', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), padding='SAME', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(100, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(26))
model.add(Activation(activation="softmax"))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 7, 7, 128)         73856     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 3, 3, 128)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1152)              0         
__________

In [13]:
batch = 500
eps = 50

opt = SGD(lr=0.01, decay=1e-5)
model.compile(opt, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=batch, epochs=eps, validation_data=(X_test, Y_test))

Train on 300000 samples, validate on 72037 samples
Epoch 1/50
300000/300000 [==============================] - 37s 125us/step - loss: 0.9581 - acc: 0.7523 - val_loss: 0.1820 - val_acc: 0.9495
Epoch 2/50
 19000/300000 [>.............................] - ETA: 32s - loss: 0.3875 - acc: 0.8911

300000/300000 [==============================] - 37s 124us/step - loss: 0.3082 - acc: 0.9134 - val_loss: 0.1312 - val_acc: 0.9649
Epoch 3/50
113500/300000 [==========>...................] - ETA: 21s - loss: 0.2432 - acc: 0.9324

300000/300000 [==============================] - 37s 124us/step - loss: 0.2306 - acc: 0.9364 - val_loss: 0.1119 - val_acc: 0.9706
Epoch 4/50
149500/300000 [=============>................] - ETA: 17s - loss: 0.1989 - acc: 0.9451

300000/300000 [==============================] - 37s 124us/step - loss: 0.1941 - acc: 0.9469 - val_loss: 0.0942 - val_acc: 0.9754
Epoch 5/50
163000/300000 [===============>..............] - ETA: 15s - loss: 0.1761 - acc: 0.9522

300000/300000 [==============================] - 37s 123us/step - loss: 0.1721 - acc: 0.9535 - val_loss: 0.0890 - val_acc: 0.9763
Epoch 6/50
168000/300000 [===============>..............] - ETA: 15s - loss: 0.1596 - acc: 0.9570

...

300000/300000 [==============================] - 38s 126us/step - loss: 0.0507 - acc: 0.9860 - val_loss: 0.0449 - val_acc: 0.9895
Epoch 47/50
170500/300000 [================>.............] - ETA: 15s - loss: 0.0484 - acc: 0.9869

300000/300000 [==============================] - 38s 125us/step - loss: 0.0487 - acc: 0.9866 - val_loss: 0.0433 - val_acc: 0.9896
Epoch 48/50
170500/300000 [================>.............] - ETA: 15s - loss: 0.0488 - acc: 0.9863

300000/300000 [==============================] - 38s 125us/step - loss: 0.0485 - acc: 0.9867 - val_loss: 0.0454 - val_acc: 0.9896
Epoch 49/50
170500/300000 [================>.............] - ETA: 15s - loss: 0.0467 - acc: 0.9868

300000/300000 [==============================] - 37s 124us/step - loss: 0.0481 - acc: 0.9866 - val_loss: 0.0428 - val_acc: 0.9902
Epoch 50/50
170500/300000 [================>.............] - ETA: 15s - loss: 0.0473 - acc: 0.9871

300000/300000 [==============================] - 37s 124us/step - loss: 0.0474 - acc: 0.9871 - val_loss: 0.0408 - val_acc: 0.9905
